In [14]:
import datetime
import time 
from polygon import RESTClient
from polygon.rest import RESTClient
from sqlalchemy import create_engine 
from sqlalchemy import text
import pandas as pd
from math import sqrt
from math import isnan
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from math import floor
from collections import defaultdict
from pycaret.regression import *

In [15]:
# pip install polygon-api-client~=1.0.0b
# !pip install --force-reinstall numpy==1.20
# pip install pycaret
# from google.colab import drive
# drive.mount('/content/drive')

In [16]:
# This funtion will lower and upper bounds when a column is sent as input
# basically used to sort FD and VOl columns
def find_fd_vol_bounds(df,col):
    n = int(df.shape[0]/3)
    df = df.sort_values(by=[col])
    low = df[col].iloc[n]
    high = df[col].iloc[2*n]
    return (low,high)

#### Classification Function : Input  is the Column name to be altered

In [17]:
# This funtion will Classify the when a column is sent as input
# basically used to sort FD and VOl columns as 1 , 2 , 3 
def replace_val(df,col,start):
    n = int(df.shape[0]/3)
    df = df.sort_values(by=[col])
    df = df.reset_index(drop=True)
    df.loc[0:n-1,col] = start+1
    df.loc[n:(2*n)-1,col] = start+2
    df.loc[2*n:3*n,col] = start+3
    df[col] = df[col].astype('int')
    return df

## To get all the input files after collecting the data for 10 hours

In [18]:
path = "./outputs/output_"
dfs = defaultdict(dict)
dfs_sort3 = defaultdict(dict)
#currency_pair_set = [("EUR","USD"),("USD","PLN")]
currency_pair_set = [("AUD","USD"),("GBP","EUR"),("USD","CAD"),("USD","JPY"),("USD","MXN"),("EUR","USD"),("USD","CNY"),("USD","CZK"),("USD","PLN"),("USD","INR")]


## Sorting method 1 , where i am sorting by two variables VOl and FD without any dependancy
#### Sorting vol first and assigning the values 1,2,3
#### Sorting FD later and assigning the values 11,12,13

In [19]:
def replace_val_1(df,col,start):
    n = int(df.shape[0]/3)
    df = df.reset_index(drop=True)
    df.loc[0:n-1,col] = start+1
    df.loc[n:(2*n)-1,col] = start+2
    df.loc[2*n:3*n,col] = start+3
    df[col] = df[col].astype('int')
    return df


for cpair in currency_pair_set:
    # Get the Currency Pair string
    cur_path = cpair[0] + cpair[1]
    cur_file_path = path + cur_path + '.csv'
    df = pd.read_csv(cur_file_path)
    df = df.loc[1:]
    # vol index : 1 , 2 , 3
    df = df.sort_values(by=['vol_val'])
    
    dfs[cur_path]['vol_low'],dfs[cur_path]['vol_high'] = find_fd_vol_bounds(df,'vol_val')
    df = replace_val_1(df,'vol_val',0)
    
    # fd index : 11 , 12 , 13
    
    dfs[cur_path]['fd_low'],dfs[cur_path]['fd_high'] = find_fd_vol_bounds(df,'fd')
    df = replace_val_1(df,'fd',10)
    
    dfs[cur_path]['df'] = df
    dfs[cur_path]['df']['return'] = dfs[cur_path]['df']['return'] * 100000
    


In [20]:
for cpair in currency_pair_set:
    # Get the Currency Pair string
    cur_path = cpair[0] + cpair[1]
    print("For currency ",cur_path)
    print(dfs[cur_path]['fd_low'],dfs[cur_path]['fd_high'])
    print(dfs[cur_path]['vol_low'],dfs[cur_path]['vol_high'])


For currency  AUDUSD
48000.00000000529 60655.73770491809
0.0006999999999999 0.000865
For currency  GBPEUR
49180.32786884355 65833.33333331622
0.0006099999999999 0.0008300000000001
For currency  USDCAD
40000.0000000044 55135.13513513128
0.00076 0.0010149999999997
For currency  USDJPY
325.581395348789 463.3204633204373
0.0919999999999845 0.1365000000000122
For currency  USDMXN
2478.929102627635 3648.802736601244
0.0125900000000029 0.0187099999999986
For currency  EURUSD
46987.95180723535 60606.06060607136
0.0007000000000001 0.0009100000000001
For currency  USDCNY
9999.999999999083 35999.99999998798
0.0001999999999995 0.0016000000000007
For currency  USDCZK
1866.6666666668489 2352.941176470708
0.0180000000000006 0.0228000000000001
For currency  USDPLN
7424.050014652693 10181.077739801349
0.0039465000000005 0.0055865000000006
For currency  USDINR
881.3559322030026 1531.2499999994886
0.027550000000005 0.0460000000000206


for each in dfs:
  data = dfs[each]['df']
  data = data.drop(['inserttime','prev_avg','stdfxrate','min_val','max_val'],axis=1)
  dfs[each]['df'] = data

## Sorting method 2 , where i am sorting by two variables VOl and FD dependently
#### FD Sorted as [21,22,23]
#### Vol Sorted as [1,2,3]

In [21]:
dfs_sort2 = defaultdict(dict)

def sortfd2(df):
    n = int(df.shape[0]/3)
    
    df = df.sort_values(by=['fd'])
    df = df.reset_index(drop=True)
    
    fd_value_lb_1 = df['fd'].iloc[n]
    fd_value_ub_2 = df['fd'].iloc[2*n]
    
    df.loc[0:n-1,'fd'] = int(21)
    df.loc[n:(2*n)-1,'fd'] = int(22)
    df.loc[2*n:3*n+1 ,'fd'] = int(23)
    
    df['fd'] = df['fd'].astype('int')
    
    print("fd low",fd_value_lb_1)
    return df,fd_value_lb_1,fd_value_ub_2

for cpair in currency_pair_set:
    
    # Get the Currency Pair string
    cur_path = cpair[0] + cpair[1]
    cur_file_path = path + cur_path + '.csv'
    
    # Read the file
    df = pd.read_csv(cur_file_path)
    # remove the top row
    df = df.loc[1:]

    
    df = df.sort_values(by=['vol_val'])
    n = int(df.shape[0]/3)
    vol_low = df['vol_val'].iloc[n]
    vol_high = df['vol_val'].iloc[2*n]
    print(vol_low,vol_high,"volllllll")
    dfs_sort2[cur_path]['vol_low'],dfs_sort2[cur_path]['vol_high'] = vol_low,vol_high
    

    df = replace_val(df,'vol_val',0)
    
    df1 = df.loc[0 : n-1]
    df2 = df.loc[n : (2*n)-1]
    df3 = df.loc[(2*n) : 3*n]
    
    
    df1,df1_fd_low,df1_fd_high = sortfd2(df1)
    df2,df2_fd_low,df2_fd_high = sortfd2(df2)
    df3,df3_fd_low,df3_fd_high = sortfd2(df3)
    
    dfs_sort2[cur_path]['df1_fd_low'] = df1_fd_low
    dfs_sort2[cur_path]['df1_fd_high'] = df1_fd_high
    
    dfs_sort2[cur_path]['df2_fd_low'] = df2_fd_low
    dfs_sort2[cur_path]['df2_fd_high'] = df2_fd_high

    dfs_sort2[cur_path]['df3_fd_low'] = df3_fd_low
    dfs_sort2[cur_path]['df3_fd_high'] = df3_fd_high
    
    #concat 3 dataframes which we split up
    newdf = pd.concat([df1,df2,df3])
    newdf = newdf.reset_index(drop=True)

    dfs_sort2[cur_path]['df'] = newdf
    dfs_sort2[cur_path]['df']['return'] = dfs_sort2[cur_path]['df']['return'] * 100000


0.0006999999999999 0.000865 volllllll
fd low 52000.000000005726
fd low 46575.34246575997
fd low 43902.439024387946
0.0006099999999999 0.0008300000000001 volllllll
fd low 58490.56603774474
fd low 44927.53623187888
fd low 47000.00000000517
0.00076 0.0010149999999997 volllllll
fd low 43902.43902437209
fd low 43157.894736841816
fd low 34123.22274881425
0.0919999999999845 0.1365000000000122 volllllll
fd low 330.93525179867265
fd low 338.58267716532885
fd low 314.2857142857449
0.0125900000000029 0.0187099999999986 volllllll
fd low 3007.51879699226
fd low 2500.000000000034
fd low 1916.308173641378
0.0007000000000001 0.0009100000000001 volllllll
fd low 53658.53658538242
fd low 43181.81818181858
fd low 42857.14285713399
0.0001999999999995 0.0016000000000007 volllllll
fd low 0.0
fd low 41999.99999994867
fd low 14285.71428571737
0.0180000000000006 0.0228000000000001 volllllll
fd low 2193.548387096356
fd low 1809.9547511313665
fd low 1782.608695652198
0.0039465000000005 0.0055865000000006 vollllll

In [22]:
# dfs_sort2[cur_path]['df1_fd_low'],dfs_sort2[cur_path]['df1_fd_high'],dfs_sort2[cur_path]['df2_fd_low'],dfs_sort2[cur_path]['df2_fd_high'],dfs_sort2[cur_path]['df3_fd_low'],dfs_sort2[cur_path]['df3_fd_high']

#### Sorting method 3 , where i am sorting by two variables FD and VOL dependently
#### FD Sorted as [1,2,3]
#### Vol Sorted as [31,32,33]

In [23]:
dfs_sort3 = defaultdict(dict)

def sortfd3(df):
    n = int(df.shape[0]/3)
    
    df = df.sort_values(by=['vol_val'])
    
    
    fd_value_lb_1 = df['vol_val'].iloc[n]
    fd_value_ub_2 = df['vol_val'].iloc[2*n]
    
    df = df.reset_index(drop=True)
    
    df.loc[0:n-1,'vol_val'] = int(31)
    df.loc[n:(2*n)-1,'vol_val'] = int(32)
    df.loc[2*n:3*n+1 ,'vol_val'] = int(33)
    
    df['vol_val'] = df['vol_val'].astype('int')



    return df,fd_value_lb_1,fd_value_ub_2

for cpair in currency_pair_set:
    
    # Get the Currency Pair string
    cur_path = cpair[0] + cpair[1]
    cur_file_path = path + cur_path + '.csv'
    
    # Read the file
    df = pd.read_csv(cur_file_path)
    # remove the top row
    df = df.loc[1:]

    
    df = df.sort_values(by=['fd'])
    n = int(df.shape[0]/3)
    vol_low = df['fd'].iloc[n]
    vol_high = df['fd'].iloc[2*n]
    
    dfs_sort3[cur_path]['fd_low'],dfs_sort3[cur_path]['fd_high'] = vol_low,vol_high

    df = replace_val(df,'fd',0)
    
    df1 = df.loc[0 : n-1]
    df2 = df.loc[n : (2*n)-1]
    df3 = df.loc[(2*n) : 3*n]
    
    
    df1,df1_fd_low,df1_fd_high = sortfd3(df1)
    df2,df2_fd_low,df2_fd_high = sortfd3(df2)
    df3,df3_fd_low,df3_fd_high = sortfd3(df3)
    
    dfs_sort3[cur_path]['df1_vol_low'] = df1_fd_low
    dfs_sort3[cur_path]['df1_vol_high'] = df1_fd_high
    
    dfs_sort3[cur_path]['df2_vol_low'] = df2_fd_low
    dfs_sort3[cur_path]['df2_vol_high'] = df2_fd_high

    dfs_sort3[cur_path]['df3_vol_low'] = df3_fd_low
    dfs_sort3[cur_path]['df3_vol_high'] = df3_fd_high
    
    #concat 3 dataframes which we split up
    newdf = pd.concat([df1,df2,df3])
    newdf = newdf.reset_index(drop=True)

    dfs_sort3[cur_path]['df'] = newdf
    dfs_sort3[cur_path]['df']['return'] = dfs_sort3[cur_path]['df']['return'] * 100000

In [24]:
dfs_sort3['EURUSD']['df'].head()

,inserttime,avgfxrate,stdfxrate,prev_avg,min_val,max_val,vol_val,fd,return
0,2022-12-06 01:30:23,1.048082,0.000081,1.048082,1.047850,1.04835,31,1,-0.130397
1,2022-12-06 03:59:14,1.049186,0.000124,1.049186,1.049010,1.04957,31,1,-34.267194
2,2022-12-06 11:29:36,1.051921,0.000097,1.051921,1.051585,1.05222,31,1,-10.528834
3,2022-12-06 07:08:24,1.051660,0.000136,1.051660,1.051425,1.05207,31,1,6.870562
4,2022-12-06 04:25:38,1.048405,0.000155,1.048405,1.048050,1.04871,31,1,12.143734


## Droping unwanted columns

In [25]:
for each in dfs:
    dfs[each]['df'] = dfs[each]['df'].drop(['inserttime','prev_avg','stdfxrate','min_val','max_val'],axis=1)
    
for each in dfs_sort2:
    dfs_sort2[each]['df'] = dfs_sort2[each]['df'].drop(['inserttime','prev_avg','stdfxrate','min_val','max_val'],axis=1)
    
for each in dfs_sort3:
    dfs_sort3[each]['df'] = dfs_sort3[each]['df'].drop(['inserttime','prev_avg','stdfxrate','min_val','max_val'],axis=1)

In [26]:
# dfs[each]['df']

In [27]:
dfs_sort3['EURUSD']['df'].head() #checking

,avgfxrate,vol_val,fd,return
0,1.048082,31,1,-0.130397
1,1.049186,31,1,-34.267194
2,1.051921,31,1,-10.528834
3,1.051660,31,1,6.870562
4,1.048405,31,1,12.143734


## Model creation for each currency type and printing the best model for each currency

#### Creating,tunning and saving the model for all 3 Sorting Methods

In [426]:
#model making
# sorting method 1
for each in dfs:
  getDataFrame = dfs[each]['df']
  clf1 = setup(data = getDataFrame, target = 'return',silent = True)
  best = compare_models(n_select = 2)
  strval = str(best[0]).split('(')[0]
  if strval == "DummyRegressor" :
    newbest = best[1]
  else:
    newbest = best[0]
  model = create_model(newbest)
  tuned_model = tune_model(model)
  model = finalize_model(tuned_model)
  save_model(model,each+"check")
  dfs[each]['best_model'] = model

for each in dfs:
  print(dfs[each]['best_model'])
    

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,9.5215,318.9948,17.8604,-0.1026,1.9682,1.0000
1,15.8634,375.7238,19.3836,-0.1934,1.9003,0.9459
2,25.7371,1461.3031,38.2270,-0.0272,2.9949,1.0000
3,16.9247,467.1672,21.6140,-0.1480,2.5811,1.0057
4,18.0894,491.2900,22.1651,-0.0031,2.3674,0.9844
5,8.7506,134.0959,11.5800,-0.0519,1.9270,1.0610
6,25.1378,1111.5148,33.3394,-0.0423,2.2709,1.1380
7,5.9740,53.6014,7.3213,-0.0125,1.5833,0.9281
8,9.8323,141.7521,11.9060,-0.0888,2.1540,0.9687


Transformation Pipeline and Model Successfully Saved
OrthogonalMatchingPursuit(fit_intercept=True, n_nonzero_coefs=1, normalize=True,
                          precompute='auto', tol=None)
BayesianRidge(alpha_1=1e-07, alpha_2=1e-07, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=False,
              lambda_1=0.01, lambda_2=0.0005, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False)
OrthogonalMatchingPursuit(fit_intercept=True, n_nonzero_coefs=1, normalize=True,
                          precompute='auto', tol=None)
ElasticNet(alpha=2.01, copy_X=True, fit_intercept=True, l1_ratio=0.022,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=2776, selection='cyclic', tol=0.0001, warm_start=False)
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)
OrthogonalMatchingPursuit(fit_intercept=True, n_nonzero_coefs=1, normalize=True,
                        

In [427]:
dfs_sort2[each]['df']

,avgfxrate,vol_val,fd,return
0,82.460980,1,21,-2.924732
1,82.425513,1,21,-6.140117
2,82.474865,1,21,4.355052
3,82.393653,1,21,-16.525847
4,82.445850,1,21,-1.358651
...,...,...,...,...
98,82.453657,3,23,9.591762
99,82.398113,3,23,33.254940
100,82.407272,3,23,1.646929
101,82.476701,3,23,17.537724


In [428]:
# sorting method 2
for each in dfs_sort2:
  getDataFrame = dfs_sort2[each]['df']
  clf1 = setup(data = getDataFrame, target = 'return',silent = True)
  best = compare_models(n_select = 2)
  strval = str(best[0]).split('(')[0]
  if strval == "DummyRegressor" :
    newbest = best[1]
  else:
    newbest = best[0]
  model = create_model(newbest)
  tuned_model = tune_model(model)
  model = finalize_model(tuned_model)
  save_model(model,each+"check2")
  dfs_sort2[each]['best_model'] = model

for each in dfs_sort2:
  print(dfs_sort2[each]['best_model'])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,8.3506,112.5254,10.6078,-0.4149,0.7639,1.1195
1,17.9959,1110.7653,33.3281,-0.1139,1.5991,2.0206
2,7.8670,140.6893,11.8613,-0.0876,1.0198,2.5098
3,9.1422,150.6152,12.2725,-0.0353,1.0171,0.6555
4,15.6767,421.3729,20.5274,-0.0092,1.5262,1.3115
5,16.4700,460.7670,21.4655,-0.0322,1.3499,1.3299
6,12.8270,368.2556,19.1900,-0.0084,1.2667,0.9423
7,20.0311,769.1447,27.7335,-0.0013,1.5266,2.5731
8,12.0454,248.2326,15.7554,-0.0043,1.1531,0.9038


Transformation Pipeline and Model Successfully Saved
LassoLars(alpha=1.0, copy_X=True, eps=0.0001, fit_intercept=True, fit_path=True,
          jitter=None, max_iter=500, normalize=True, positive=False,
          precompute='auto', random_state=7320, verbose=False)
PassiveAggressiveRegressor(C=0.182, average=False, early_stopping=False,
                           epsilon=0.7, fit_intercept=False,
                           loss='squared_epsilon_insensitive', max_iter=1000,
                           n_iter_no_change=5, random_state=1079, shuffle=True,
                           tol=0.001, validation_fraction=0.1, verbose=0,
                           warm_start=False)
LassoLars(alpha=1.0, copy_X=True, eps=0.02, fit_intercept=True, fit_path=True,
          jitter=None, max_iter=500, normalize=True, positive=False,
          precompute='auto', random_state=2751, verbose=False)
ElasticNet(alpha=4.68, copy_X=True, fit_intercept=True,
           l1_ratio=0.957999999999999, max_iter=1000, no

In [355]:
# sorting method 3

for each in dfs_sort3:
  getDataFrame = dfs_sort3[each]['df']
  clf1 = setup(data = getDataFrame, target = 'return',silent = True)
  best = compare_models(n_select = 2)
  strval = str(best[0]).split('(')[0]
  if strval == "DummyRegressor" :
    newbest = best[1]
  else:
    newbest = best[0]
  model = create_model(newbest)
  tuned_model = tune_model(model)
  model = finalize_model(tuned_model)
  save_model(model,each+"check3")
  dfs_sort3[each]['best_model'] = model

for each in dfs_sort3:
  print(dfs_sort3[each]['best_model'])
    

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,28.0430,1644.4715,40.5521,-0.0648,1.5426,4.6015
1,5.7565,70.5637,8.4002,0.2660,0.7392,1.2616
2,10.1768,175.1189,13.2333,-0.7763,1.5573,1.0189
3,30.2944,1635.9316,40.4467,-0.4321,1.6085,2.7037
4,14.8004,317.7838,17.8265,0.0039,1.3835,1.1125
5,22.2893,927.4810,30.4546,0.0217,1.5345,1.3073
6,10.4895,133.8704,11.5702,-0.1110,1.8449,6.5089
7,17.3134,517.8157,22.7556,-0.1815,1.7123,1.0110
8,12.1674,349.5104,18.6952,0.0330,1.7151,1.3019


Transformation Pipeline and Model Successfully Saved
LassoLars(alpha=1.0, copy_X=True, eps=1e-05, fit_intercept=True, fit_path=True,
          jitter=None, max_iter=500, normalize=True, positive=False,
          precompute='auto', random_state=2122, verbose=False)
BayesianRidge(alpha_1=0.001, alpha_2=0.0001, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=False,
              lambda_1=0.005, lambda_2=1e-07, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False)
LassoLars(alpha=1.0, copy_X=True, eps=0.007, fit_intercept=True, fit_path=True,
          jitter=None, max_iter=500, normalize=True, positive=False,
          precompute='auto', random_state=1525, verbose=False)
BayesianRidge(alpha_1=0.0001, alpha_2=0.15, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=False,
              lambda_1=0.1, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False

In [356]:
best[0]

HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True, max_iter=100,
               tol=1e-05, warm_start=False)

In [359]:
print(best)

[HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True, max_iter=100,
               tol=1e-05, warm_start=False), LassoLars(alpha=1.0, copy_X=True, eps=2.220446049250313e-16, fit_intercept=True,
          fit_path=True, jitter=None, max_iter=500, normalize=True,
          positive=False, precompute='auto', random_state=3620, verbose=False)]


#### Creating and saving the model for all 3 Sorting Methods

## Replacing the real time data with classification
#### Sorting method 1 will calculateModelReturn_Model1()
#### Sorting method 2 will calculateModelReturn_Model2()
#### Sorting method 3 will calculateModelReturn_Model3()

In [32]:
def calculateModelReturn_model1(curr,avgfxrate,vol_val,fd):
    each = curr[0] + curr[1]
    
    #replacing fd and vol with classifiers and labels
    print("\n\n\nSorting method 1 Prediction")
    print("before : : vol    ",vol_val," fd    ",fd)
   
    print(dfs[each]['vol_low'])
    print(dfs[each]['vol_high'])
    print(dfs[each]['fd_low'])
    print(dfs[each]['fd_high'])
    
    if fd < dfs[each]['fd_low']:
        fd = 11
    elif fd > dfs[each]['fd_low'] and fd <= dfs[each]['fd_high']:
        fd = 12
    else:
        fd = 13
    
    if vol_val < dfs[each]['vol_low']:
        vol_val = 1
    elif vol_val > dfs[each]['vol_low'] and fd <= dfs[each]['vol_high']:
        vol_val = 2
    else:
        vol_val = 3
        
        
    #creating a dataframe from real time data to feed into the model for prediction
    data = [[avgfxrate, vol_val,fd]]
    df_predict = pd.DataFrame(data, columns=['avgfxrate', 'vol_val','fd'])
    
    #predicting return from saved model
    loaded_model = load_model(each+"check")
    predictions = predict_model(loaded_model, data = df_predict)
    print(str(loaded_model))
    
    newdf = predictions['Label']
    val = newdf._get_value(0, 'Label')
    
    return val

In [33]:
def calculateModelReturn_model2(curr,avgfxrate,vol_val,fd):
    each = curr[0] + curr[1]
    print("\n\n\nSorting method 2 Prediction")
    print("before : : vol    ",vol_val," fd    ",fd)
    
    print(dfs_sort2[each]['vol_low'],dfs_sort2[each]['vol_high'] )
    print(dfs_sort2[each]['df1_fd_low'],dfs_sort2[each]['df1_fd_high']) 
    print(dfs_sort2[each]['df2_fd_low'] , dfs_sort2[each]['df2_fd_high'])
    print(dfs_sort2[each]['df3_fd_low'] , dfs_sort2[each]['df3_fd_high'] ) 
                                                                        
    def findrange(fd_range,cur,val):
        fd_key = "df"+ str(fd_range) +"_fd_"
        if val > dfs_sort2[cur][fd_key+"high"]:
            return 23
        elif val > dfs_sort2[cur][fd_key+"low"]:
            return 22
        else:
            return 21
                                                                        
    if vol_val > dfs_sort2[each]['vol_high']:
        vol_val = 3
        fd = findrange(3,each,fd)
    elif vol_val > dfs_sort2[each]['vol_low']:
        vol_val = 2
        fd = findrange(2,each,fd)
    else:
        vol_val = 1
        fd = findrange(2,each,fd)
    
    #creating a dataframe from real time data to feed into the model for prediction
    data = [[avgfxrate, vol_val,fd]]
    df_predict = pd.DataFrame(data, columns=['avgfxrate', 'vol_val','fd'])
    
    #predicting return from saved model
    loaded_model2 = load_model(each+"check2")
    predictions2 = predict_model(loaded_model2, data = df_predict)
    print(str(loaded_model2))
    
    newdf = predictions2['Label']
    val = newdf._get_value(0, 'Label')
    
    return val

In [34]:
def calculateModelReturn_model3(curr,avgfxrate,vol_val,fd):
    each = curr[0] + curr[1]
    print("\n\n\nSorting method 3 Prediction")
    print("Values Before prediction: : vol    ",vol_val," fd    ",fd)
    
    print(dfs_sort3[each]['fd_low'],dfs_sort3[each]['fd_high'] )
    print(dfs_sort3[each]['df1_vol_low'],dfs_sort3[each]['df1_vol_high']) 
    print(dfs_sort3[each]['df2_vol_low'] , dfs_sort3[each]['df2_vol_high'])
    print(dfs_sort3[each]['df3_vol_low'] , dfs_sort3[each]['df3_vol_high'] ) 
                                                                     
    def findrange(fd_range,cur,val):
        vol_key = "df"+ str(fd_range) +"_vol_"
        if val > dfs_sort3[cur][vol_key+"high"]:
            return 33
        elif val > dfs_sort3[cur][vol_key+"low"]:
            return 32
        else:
            return 31
                                                                        
    if fd > dfs_sort3[each]['fd_high']:
        fd = 3
        vol_val = findrange(3,each,vol_val)
    elif fd > dfs_sort3[each]['fd_low']:
        fd = 2
        vol_val = findrange(2,each,vol_val)
    else:
        fd = 1
        vol_val = findrange(2,each,vol_val)
    
    #creating a dataframe from real time data to feed into the model for prediction
    data = [[avgfxrate, vol_val,fd]]
    df_predict = pd.DataFrame(data, columns=['avgfxrate', 'vol_val','fd'])
    
    #predicting return from saved model
    loaded_model3 = load_model(each+"check3")
    predictions3 = predict_model(loaded_model3, data = df_predict)
    print(str(loaded_model3))
    
    newdf = predictions3['Label']
    val = newdf._get_value(0, 'Label')

    return val

In [35]:
a = calculateModelReturn_model1(['EUR', 'USD',1],1.072223,0.00078,2)
#checking for sample input




Sorting method 1 Prediction
before : : vol     0.00078  fd     2
0.0007000000000001
0.0009100000000001
46987.95180723535
60606.06060607136
Transformation Pipeline and Model Successfully Loaded
Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(display_types=False,
                                      ml_usecase='regression',
                                      target='return')),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strategy='mean',
                                target_variable=None)),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_strategy='least...
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
                ('dummy', Dumm

#### Assignment 3 code is added here to run it every 6 seconds

In [36]:
class portfolio(object):
    def __init__(self,from_,to):
        # Initialize the 'From' currency amont to 1
        self.from_ = from_
        self.to = to
        # We want to keep track of state, to see what our next trade should be
        self.Prev_Action_was_Buy = False
    
    # This defines a function to buy the 'To' currency. It will always buy the max amount, in whole number
    # increments
    def buy_curr(self, price, num_to_buy, ret_loss, amount, curr2):
        print("buy_cur")
        if amount >= 1 and amount <=100:
            amount -= num_to_buy
            self.Prev_Action_was_Buy = True
            curr2 += num_to_buy*price
            print("Bought %d worth of the target currency (%s). Our current profits and losses in the original currency (%s) are: %f." % (num_to_buy,self.to,self.from_,(amount-1)))
        else:
            print("There was not enough of the original currency (%s) to make another buy." % self.from_)
        f = open('balances_'+str(self.from_)+str(self.to)+'.csv', 'a', newline='')
        writer = csv.writer(f)
        # ['Balances', 'prof_loss', 'return_pf']
        writer.writerow([num_to_buy, amount-1, ret_loss])
        f.close()
    # This defines a function to sell the 'To' currency. It will always sell the max amount, in a whole number
    # increments
    def sell_curr(self, price, num_to_sell, ret_loss, amount, curr2):
        if curr2 >= 1 and curr2 <=100:
            amount += num_to_sell * (1/price)
            self.Prev_Action_was_Buy = False
            curr2 -= num_to_sell
            print("Sold %d worth of the target currency (%s). Our current profits and losses in the original currency (%s) are: %f." % (num_to_sell,self.to,self.from_,(amount-1)))
        else:
            print("There was not enough of the target currency (%s) to make another sell." % self.to)   
        f = open('balances_'+str(self.from_)+str(self.to)+'.csv', 'a', newline='')
        # ['Balances', 'prof_loss', 'return_pf']
        writer = csv.writer(f)
        writer.writerow([num_to_sell, amount-1, ret_loss])
        f.close()

In [37]:
# getting key from library
from Auth import token
auth = Token()
keyValue = auth.get_key()

hey


In [ ]:
import csv
def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M:%S')

def reset_raw_data_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("DROP TABLE "+curr[0]+curr[1]+"_raw;"))
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_raw(ticktime text, fxrate  numeric, inserttime text);"))

# This creates a table for storing the raw, unaggregated price data for each currency pair in the SQLite database
def initialize_raw_data_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_raw(ticktime text, fxrate  numeric, inserttime text);"))

# This will create table to store output of 3 sorting methods
def initialize_return_data_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_returnModel1(ticktime text, actual  numeric,predicted  numeric, error numeric);"))
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_returnModel2(ticktime text, actual  numeric,predicted  numeric, error numeric);"))
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_returnModel3(ticktime text, actual  numeric,predicted  numeric, error numeric);"))

# This creates a table for storing the (6 min interval) aggregated price data for each currency pair in the SQLite database            
def initialize_aggregated_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_agg(inserttime text, avgfxrate numeric, stdfxrate numeric, prev_avg numeric, min_val numeric, max_val numeric, vol_val numeric, fd numeric, return numeric);"))

# This function is called every 6 minutes to aggregate the data, store it in the aggregate table, 
# and then delete the raw data
def aggregate_raw_data_tables(engine,currency_pairs, count):
    with engine.begin() as conn:
        for curr in currency_pairs:
            result = conn.execute(text("SELECT AVG(fxrate) as avg_price, COUNT(fxrate) as tot_count FROM "+curr[0]+curr[1]+"_raw;"))

            for row in result:
                avg_price = row.avg_price
                tot_count = row.tot_count
            std_res = conn.execute(text("SELECT SUM((fxrate - "+str(avg_price)+")*(fxrate - "+str(avg_price)+"))/("+str(tot_count)+"-1) as std_price FROM "+curr[0]+curr[1]+"_raw;"))
            for row in std_res:
                std_price = sqrt(row.std_price)
            date_res = conn.execute(text("SELECT MAX(ticktime) as last_date FROM "+curr[0]+curr[1]+"_raw;"))
            for row in date_res:
                last_date = row.last_date
                
            values = conn.execute(text("SELECT AVG(fxrate) as avg_price, MIN(fxrate) as min_price, MAX(fxrate) as max_price, MAX(fxrate)-MIN(fxrate) as vol FROM "+curr[0]+curr[1]+"_raw;"))
            for vls in values: 
                avg_price1 = vls.avg_price
                min_price = vls.min_price
                max_price = vls.max_price
                vol = vls.vol
                
            # During thr first period (first 6 min) fd is None
            if count <= 360:
                fd = None
                ret = 0
            
            # FOr periods 2 to 100 fetch values and calculate fd and update values
            if count > 360:
                #Get the number of the last entry of the aggregate table
                cnt = conn.execute(text("SELECT COUNT(prev_avg) as cnts FROM "+curr[0]+curr[1]+"_agg;"))
                for cn in cnt:
                    cnts = cn.cnts
                #Fetch the previous average and volatility values using the number of the last row
                prev_vals = conn.execute(text("SELECT prev_avg, vol_val FROM "+curr[0]+curr[1]+"_agg LIMIT "+str(cnts)+"-10, 10;"))

                for vls1 in prev_vals:
                    prev_avg = vls1.prev_avg
                    prev_vol = vls1.vol_val
                
                ret = (avg_price1 - prev_avg)/prev_avg
                print("2:aggregate_raw_data_tables :: return ",ret,"avg_price1",avg_price1,"prev_avg",prev_avg)
                #Calculate keltner bands using fetched previous values
                prev_upp_kel_band = [prev_avg + n*0.025*prev_vol for n in range(1, 101)]
                prev_loww_kel_band = [prev_avg - n*0.025*prev_vol for n in range(1, 101)]
                
                #Calculate fractral dimension using the bands
                fd = 0
                fd_count = sum(ii < max_price for ii in prev_upp_kel_band) + sum(ii > min_price for ii in prev_loww_kel_band)
#                 print("fd_count:", fd_count)

                if vol != 0:
                    fd = fd_count/vol
#                     print("fd:", fd)
            
            #Save the values into the database table
            conn.execute(text("INSERT INTO "+curr[0]+curr[1]+"_agg(inserttime, avgfxrate, stdfxrate, prev_avg, min_val, max_val, vol_val, fd, return) VALUES (:inserttime, :avgfxrate, :stdfxrate, :prev_avg, :min_val, :max_val, :vol_val, :fd, :return);"),[{"inserttime": last_date, "avgfxrate": avg_price, "stdfxrate": std_price, "prev_avg": avg_price1, "min_val": min_price, "max_val": max_price, "vol_val": vol, "fd": fd, "return": ret}])
            
            #_return(ticktime text, actual  numeric,predicted  numeric, error numeric);
#             ret = ret * 1000000
            predicted_return1 = 0
            predicted_return2 = 0
            predicted_return3 = 0
            err = 0
            err1,err2,err3 = 0,0,0
            
            if count > 360:
                predicted_return1 = calculateModelReturn_model1(curr,avg_price,vol,fd)
                predicted_return1 = float(predicted_return1/100000)
                
                predicted_return2 = calculateModelReturn_model2(curr,avg_price,vol,fd)
                predicted_return2 = float(predicted_return2/100000)
                
                predicted_return3 = calculateModelReturn_model3(curr,avg_price,vol,fd)
                predicted_return3 = float(predicted_return3/100000)
                
            
            err1 = ret - predicted_return1
            err2 = ret - predicted_return2
            err3 = ret - predicted_return3
            
            
            print("currency pair",curr[0]+curr[1],":","count :  ",count,"predicted_return1 : ",predicted_return1,"    Actual return :  ",ret,"     error1 :" ,err1 )
            print("currency pair",curr[0]+curr[1],":","count :  ",count,"predicted_return2 : ",predicted_return2,"    Actual return :  ",ret,"     error2 :" ,err2 )
            print("currency pair",curr[0]+curr[1],":","count :  ",count,"predicted_return3 : ",predicted_return3,"    Actual return :  ",ret,"     error3 :" ,err3 )
            
            conn.execute(text("INSERT INTO "+curr[0]+curr[1]+"_returnModel1(ticktime,actual,predicted,error) VALUES (:ticktime, :actual ,:predicted ,:error);"),[{"ticktime": last_date, "actual": ret, "predicted": predicted_return1, "error": err1}])
            conn.execute(text("INSERT INTO "+curr[0]+curr[1]+"_returnModel2(ticktime,actual,predicted,error) VALUES (:ticktime, :actual ,:predicted ,:error);"),[{"ticktime": last_date, "actual": ret, "predicted": predicted_return2, "error": err2}])
            conn.execute(text("INSERT INTO "+curr[0]+curr[1]+"_returnModel3(ticktime,actual,predicted,error) VALUES (:ticktime, :actual ,:predicted ,:error);"),[{"ticktime": last_date, "actual": ret, "predicted": predicted_return3, "error": err3}])
            

def main(currency_pairs):
    # The api key given by the professor
    # key = key_fun() # Fetch the API key from the library
    key = keyValue
    for currency in currency_pairs:
        # Set the input variables to the API
        from_ = currency[0]
        to = currency[1]
    
        f = open('.balances_'+str(from_)+str(to)+'.csv', 'w', newline='')
        writer = csv.DictWriter(f, fieldnames=['Balances', 'prof_loss', 'return_pf'])
        writer.writeheader()
        
        # writer.writerow(['Balances', 'prof_loss', 'return_pf'])
        f.close()
    # Number of list iterations - each one should last about 1 second
    count = 0
    agg_count = 0
    
    # Create an engine to connect to the database; setting echo to false should stop it from logging in std.out
    engine = create_engine("sqlite+pysqlite:///sqlite/final.db", echo=False, future=True)
    
    # Create the needed tables in the database
    initialize_raw_data_tables(engine,currency_pairs)
    initialize_aggregated_tables(engine,currency_pairs)
    initialize_return_data_tables(engine,currency_pairs)
    
    # Open a RESTClient for making the api calls
    client = RESTClient(key)
    # with RESTClient(key) as client:
        # Loop that runs until the total duration of the program hits 24 hours. 
    while count <= 36000: # 86400 seconds = 24 hours
        # Make a check to see if 6 minutes has been reached or not
        if agg_count == 360:
            print("\n 6mins Timer is up : ", count)
            # Aggregate the data and clear the raw data tables
            aggregate_raw_data_tables(engine,currency_pairs, count)
            
            reset_raw_data_tables(engine,currency_pairs)
            agg_count = 0
        
        # Only call the api every 1 second, so wait here for 0.75 seconds, because the 
        # code takes about .15 seconds to run
        time.sleep(0.6)
        
        # Increment the counters
        count += 1
        agg_count +=1

        # Loop through each currency pair
        for currency in currency_pairs:
            # Set the input variables to the API
            from_ = currency[0]
            to = currency[1]

            # Call the API with the required parameters
            try:
                resp = client.get_real_time_currency_conversion(from_, to, amount=100, precision=2)
    
            except:
                continue

            # This gets the Last Trade object defined in the API Resource
            last_trade = resp.last
            
            # Format the timestamp from the result
            dt = ts_to_datetime(last_trade.timestamp)

            # Get the current time and format it
            insert_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            # Calculate the price by taking the average of the bid and ask prices
            avg_price = (last_trade.bid + last_trade.ask)/2

            # Write the data to the SQLite database, raw data tables
            with engine.begin() as conn:
                conn.execute(text("INSERT INTO "+from_+to+"_raw(ticktime, fxrate, inserttime) VALUES (:ticktime, :fxrate, :inserttime)"),[{"ticktime": dt, "fxrate": avg_price, "inserttime": insert_time}])
                


currency_pairs = [["AUD","USD",[],portfolio("AUD","USD"), -1],
                  ["GBP","EUR",[],portfolio("GBP","EUR"), -1],
                  ["USD","CAD",[],portfolio("USD","CAD"), -1],
                  ["USD","JPY",[],portfolio("USD","JPY"), -1],
                  ["USD","MXN",[],portfolio("USD","MXN"), -1],
                  ["EUR","USD",[],portfolio("EUR","USD"), 1],
                  ["USD","CNY",[],portfolio("USD","CNY"), 1],
                  ["USD","CZK",[],portfolio("USD","CZK"), 1],
                  ["USD","PLN",[],portfolio("USD","PLN"), 1],
                  ["USD","INR",[],portfolio("USD","INR"), 1]]



#Run the main data collection loop
main(currency_pairs)  
    